In [24]:
import numpy as np

from settings import PROCESSED_DATA_DIR
import pandas as pd
from pathlib import Path
from reduce_memory_usage import reduce_memory_usage
import tensorflow as tf
from tensorflow import keras
tf.get_logger().setLevel('INFO')

In [16]:
df = pd.read_csv(Path(PROCESSED_DATA_DIR, 'bundesliga_matches.csv'))
df = reduce_memory_usage(df)
df['info_date'] = pd.to_datetime(df['info_date'])
df = df.sort_values(by=['info_date'])

Memory usage of dataframe is 4.627826690673828 MB
Memory usage of dataframe after reduction 1.2287025451660156 MB
Reduced by 73.44968540757709 % 


In [17]:
predictors = [c for c in df.columns if c.startswith('feat_')]
cutoff_date = '2023-05-01'

train_features = df[df['info_date'] < cutoff_date][predictors]
train_target = df[df['info_date'] < cutoff_date]['target']

test_features = df[df['info_date'] >= cutoff_date][predictors]
test_target = df[df['info_date'] >= cutoff_date]['target']

In [18]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=[len(train_features.keys())]),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [19]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Precision(name='precision')])

In [20]:
model.fit(train_features, train_target, epochs=50)

Epoch 1/50
111/111 [==============================] - 1s 2ms/step - loss: 52.1630 - precision: 0.3666
Epoch 2/50
111/111 [==============================] - 0s 2ms/step - loss: 17.7769 - precision: 0.3994
Epoch 3/50
111/111 [==============================] - 0s 2ms/step - loss: 17.9987 - precision: 0.3988
Epoch 4/50
111/111 [==============================] - 0s 2ms/step - loss: 22.7140 - precision: 0.4006
Epoch 5/50
111/111 [==============================] - 0s 2ms/step - loss: 21.7019 - precision: 0.4088
Epoch 6/50
111/111 [==============================] - 0s 2ms/step - loss: 20.3828 - precision: 0.3954
Epoch 7/50
111/111 [==============================] - 0s 2ms/step - loss: 14.4884 - precision: 0.4020
Epoch 8/50
111/111 [==============================] - 0s 2ms/step - loss: 17.3748 - precision: 0.4011
Epoch 9/50
111/111 [==============================] - 0s 2ms/step - loss: 18.4424 - precision: 0.4067
Epoch 10/50
111/111 [==============================] - 0s 2ms/step - loss: 14.4867

In [22]:
test_loss, test_acc = model.evaluate(test_features,  test_target, verbose=2)

10/10 - 0s - loss: 2.6148 - precision: 0.6154 - 22ms/epoch - 2ms/step


In [29]:
prediction = model.predict(np.array(test_features))
# show prediction in either 0 or 1
prediction = np.where(prediction > 0.5, 1, 0)
prediction = pd.Series(prediction.flatten(), index=test_target.index)
prediction

10/10 [==============================] - 0s 894us/step


2225    0
2440    0
477     1
3112    0
1843    0
       ..
708     1
494     1
3296    1
2242    0
2457    1
Length: 316, dtype: int64